# Kabu-Station API exercise

- API document: https://kabucom.github.io/kabusapi/ptal/index.html
- API reference: https://kabucom.github.io/kabusapi/reference/index.html

## 0. Imports

In [53]:
import urllib.request
import json
import pandas as pd
pd.set_option('display.max_columns', 0)
import pprint
pp = pprint.PrettyPrinter(indent=4)

BASEURL = 'http://localhost:18080/kabusapi/'
BASEURL_TEST = 'http://localhost:18081/kabusapi/'

## 1. Get Token

In [58]:
from getpass import getpass
password = getpass()

 ······


In [34]:
json_data = json.dumps({'APIPassword': password}).encode('utf8')
req = urllib.request.Request(
    BASEURL+'token', 
    json_data, 
    method='POST'
)
req.add_header('Content-Type', 'application/json')

res = urllib.request.urlopen(req)
print(res.status, res.reason)

content = json.loads(res.read())
token = content['Token']

200 OK


## 2. Get Push market price

about PUSH: https://kabucom.github.io/kabusapi/ptal/push.html

In [8]:
import sys
import websocket
import _thread

def on_message(ws, message):
    print('--- RECV MSG. --- ')
    print(message)

def on_error(ws, error):
    print('--- ERROR --- ')
    print(error)

def on_close(ws):
    print('--- DISCONNECTED --- ')

def on_open(ws):
    print('--- CONNECTED --- ')
    def run(*args):
        while(True):
            line = sys.stdin.readline()
            if line != '':
                print('closing...')
                ws.close()
    #_thread.start_new_thread(run, ())

url = 'ws://localhost:18080/kabusapi/websocket'
# websocket.enableTrace(True)
ws = websocket.WebSocketApp(
    url,
    on_message = on_message,
    on_error = on_error,
    on_close = on_close
)
ws.on_open = on_open
#ws.run_forever()

## 3. Get market price board info

In [39]:
brand_code = '4689@1'

req = urllib.request.Request(BASEURL+'board/'+brand_code, method='GET')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', token)

res = urllib.request.urlopen(req)
print(res.status, res.reason)
#for header in res.getheaders():
#      print(header)

content = json.loads(res.read())

pp.pprint(content)

print(content['CurrentPriceTime'], content['CurrentPrice'])
print(content['OpeningPriceTime'], content['OpeningPrice'])
print(content['HighPriceTime'], content['HighPrice'])
print(content['LowPriceTime'], content['LowPrice'])
print(content['OpeningPrice'])
#for i in range(1, 10):
#    print(content['Sell'+str(i)])
#for i in range(1, 10):
#    print(content['Buy'+str(i)])

200 OK
{   'AskPrice': 740.3,
    'AskQty': 1600.0,
    'AskSign': '0101',
    'AskTime': '2021-09-14T15:00:00+09:00',
    'BidPrice': 741.0,
    'BidQty': 14600.0,
    'BidSign': '0101',
    'BidTime': '2021-09-14T15:00:00+09:00',
    'Buy1': {   'Price': 740.3,
                'Qty': 1600.0,
                'Sign': '0101',
                'Time': '2021-09-14T15:00:00+09:00'},
    'Buy10': {'Price': 739.4, 'Qty': 38600.0},
    'Buy2': {'Price': 740.2, 'Qty': 1300.0},
    'Buy3': {'Price': 740.1, 'Qty': 1300.0},
    'Buy4': {'Price': 740.0, 'Qty': 2400.0},
    'Buy5': {'Price': 739.9, 'Qty': 1300.0},
    'Buy6': {'Price': 739.8, 'Qty': 1300.0},
    'Buy7': {'Price': 739.7, 'Qty': 2100.0},
    'Buy8': {'Price': 739.6, 'Qty': 1800.0},
    'Buy9': {'Price': 739.5, 'Qty': 31500.0},
    'CalcPrice': 740.9,
    'ChangePreviousClose': 36.9,
    'ChangePreviousClosePer': 5.24,
    'CurrentPrice': 740.9,
    'CurrentPriceChangeStatus': '0061',
    'CurrentPriceStatus': 1,
    'CurrentPriceTime'

In [54]:
#df = pd.DataFrame( list(content.items()) )
df = pd.DataFrame.from_dict(content)
df

,OverSellQty,UnderBuyQty,TotalMarketValue,MarketOrderSellQty,MarketOrderBuyQty,BidTime,AskTime,Exchange,ExchangeName,TradingVolume,TradingVolumeTime,VWAP,TradingValue,BidQty,BidPrice,BidSign,Sell1,Sell2,Sell3,Sell4,Sell5,Sell6,Sell7,Sell8,Sell9,Sell10,AskQty,AskPrice,AskSign,Buy1,Buy2,Buy3,Buy4,Buy5,Buy6,Buy7,Buy8,Buy9,Buy10,Symbol,SymbolName,CurrentPrice,CurrentPriceTime,CurrentPriceChangeStatus,CurrentPriceStatus,CalcPrice,PreviousClose,PreviousCloseTime,ChangePreviousClose,ChangePreviousClosePer,OpeningPrice,OpeningPriceTime,HighPrice,HighPriceTime,LowPrice,LowPriceTime,SecurityType
Price,5488100.0,3184600.0,5.672564e+12,0.0,0.0,2021-09-14T15:00:00+09:00,2021-09-14T15:00:00+09:00,1,東証１部,37270900.0,2021-09-14T15:00:00+09:00,731.9737,2.728132e+10,14600.0,741.0,0101,741.0,741.1,741.2,741.3,741.4,741.5,741.6,741.7,741.8,741.9,1600.0,740.3,0101,740.3,740.2,740.1,740.0,739.9,739.8,739.7,739.6,739.5,739.4,4689,Ｚホールディングス,740.9,2021-09-14T15:00:00+09:00,0061,1,740.9,704.0,2021-09-13T00:00:00+09:00,36.9,5.24,704.0,2021-09-14T09:00:00+09:00,744.4,2021-09-14T13:56:58+09:00,702.5,2021-09-14T09:00:05+09:00,1
Qty,5488100.0,3184600.0,5.672564e+12,0.0,0.0,2021-09-14T15:00:00+09:00,2021-09-14T15:00:00+09:00,1,東証１部,37270900.0,2021-09-14T15:00:00+09:00,731.9737,2.728132e+10,14600.0,741.0,0101,14600.0,2300.0,5400.0,8500.0,27600.0,7800.0,4400.0,7800.0,117800.0,3000.0,1600.0,740.3,0101,1600.0,1300.0,1300.0,2400.0,1300.0,1300.0,2100.0,1800.0,31500.0,38600.0,4689,Ｚホールディングス,740.9,2021-09-14T15:00:00+09:00,0061,1,740.9,704.0,2021-09-13T00:00:00+09:00,36.9,5.24,704.0,2021-09-14T09:00:00+09:00,744.4,2021-09-14T13:56:58+09:00,702.5,2021-09-14T09:00:05+09:00,1
Sign,5488100.0,3184600.0,5.672564e+12,0.0,0.0,2021-09-14T15:00:00+09:00,2021-09-14T15:00:00+09:00,1,東証１部,37270900.0,2021-09-14T15:00:00+09:00,731.9737,2.728132e+10,14600.0,741.0,0101,0101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1600.0,740.3,0101,0101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4689,Ｚホールディングス,740.9,2021-09-14T15:00:00+09:00,0061,1,740.9,704.0,2021-09-13T00:00:00+09:00,36.9,5.24,704.0,2021-09-14T09:00:00+09:00,744.4,2021-09-14T13:56:58+09:00,702.5,2021-09-14T09:00:05+09:00,1
Time,5488100.0,3184600.0,5.672564e+12,0.0,0.0,2021-09-14T15:00:00+09:00,2021-09-14T15:00:00+09:00,1,東証１部,37270900.0,2021-09-14T15:00:00+09:00,731.9737,2.728132e+10,14600.0,741.0,0101,2021-09-14T15:00:00+09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1600.0,740.3,0101,2021-09-14T15:00:00+09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4689,Ｚホールディングス,740.9,2021-09-14T15:00:00+09:00,0061,1,740.9,704.0,2021-09-13T00:00:00+09:00,36.9,5.24,704.0,2021-09-14T09:00:00+09:00,744.4,2021-09-14T13:56:58+09:00,702.5,2021-09-14T09:00:05+09:00,1


## 4. Get brand info

In [37]:
brand_code = '4689@1'

req = urllib.request.Request(BASEURL+'symbol/'+brand_code, method='GET')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', token)

res = urllib.request.urlopen(req)
print(res.status, res.reason)
#for header in res.getheaders():
#      print(header)

content = json.loads(res.read())
pp.pprint(content)

200 OK
{   'BisCategory': '5250      ',
    'DisplayName': 'ＺＨＤ',
    'Exchange': 1,
    'ExchangeName': '東証１部',
    'FiscalYearEndBasic': 20220331,
    'KCMarginBuy': True,
    'KCMarginSell': True,
    'LowerLimit': 590.9,
    'MarginBuy': True,
    'MarginSell': True,
    'PriceRangeGroup': '10003',
    'Symbol': '4689',
    'SymbolName': 'Ｚホールディングス',
    'TotalMarketValue': 5672563853885.5,
    'TotalStocks': 7655201.0,
    'TradingUnit': 100.0,
    'UpperLimit': 890.9}
